This is optional in case needs to connect to previous module can use it

In [1]:
# check langchain version it should be 0.3 if not uncomment below pip command to install the correct version
import langchain, langchain_community
print(langchain.__version__)
print(langchain_community.__version__)
# !pip install langchain==0.3.27 langchain-openai==0.3.33 langchain-community==0.3.24

0.3.27
0.3.24


In [3]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.tools import tool

# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
# The LLM 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch, RunnableLambda
import random
from datetime import datetime

# Enhanced tool functionalities with more realistic responses
def calculate_math(expression):
    """Enhanced math calculation with actual computation"""
    try:
        # Safe evaluation of common math expressions
        expression_clean = expression.lower().replace('what\'s', '').replace('what is', '').strip('? ')
        
        # Map words to operators
        word_to_operator = {
            'plus': '+', 'add': '+', 'sum': '+',
            'minus': '-', 'subtract': '-', 'difference': '-',
            'times': '*', 'multiply': '*', 'product': '*', 'x': '*',
            'divided by': '/', 'divide': '/', 'over': '/',
            'modulo': '%', 'mod': '%',
            'power': '**', 'to the power of': '**'
        }
        
        # Replace words with operators
        for word, operator in word_to_operator.items():
            expression_clean = expression_clean.replace(word, operator)
        
        # Safe evaluation
        result = eval(expression_clean, {"__builtins__": {}}, {})
        
        explanations = [
            f"Let me calculate that for you: {expression_clean} = {result}",
            f"The answer to {expression_clean} is {result}",
            f"After calculating: {expression_clean} = {result}"
        ]
        return random.choice(explanations)
        
    except:
        explanations = [
            f"I understand you want to calculate: '{expression}'. While I can't evaluate complex expressions safely, here's what I suggest: break it down into smaller steps or use a calculator for precise results.",
            f"For the calculation '{expression}', I recommend using a dedicated math tool or calculator for accurate results."
        ]
        return random.choice(explanations)

def get_weather(location):
    """Enhanced weather simulation with realistic data"""
    locations_weather = {
        "new york": {"temp": random.randint(65, 85), "condition": "Partly Cloudy", "humidity": "65%"},
        "london": {"temp": random.randint(50, 65), "condition": "Rainy", "humidity": "85%"},
        "tokyo": {"temp": random.randint(70, 90), "condition": "Sunny", "humidity": "70%"},
        "paris": {"temp": random.randint(60, 75), "condition": "Cloudy", "humidity": "75%"},
        "sydney": {"temp": random.randint(75, 95), "condition": "Sunny", "humidity": "60%"},
        "dubai": {"temp": random.randint(85, 105), "condition": "Clear", "humidity": "45%"}
    }
    
    location_lower = location.lower()
    for loc, data in locations_weather.items():
        if loc in location_lower:
            weather_info = data
            break
    else:
        # Default weather for unknown locations
        weather_info = {
            "temp": random.randint(60, 80), 
            "condition": random.choice(["Sunny", "Cloudy", "Partly Cloudy"]),
            "humidity": f"{random.randint(50, 80)}%"
        }
    
    responses = [
        f"Current weather in {location.title()}: {weather_info['temp']}°F, {weather_info['condition']}, Humidity: {weather_info['humidity']}. Perfect time to enjoy the outdoors!",
        f"Weather update for {location.title()}: {weather_info['condition']} with temperatures around {weather_info['temp']}°F. Humidity is {weather_info['humidity']}. Dress accordingly!",
        f"Here's the weather in {location.title()}: {weather_info['temp']}°F and {weather_info['condition'].lower()}. Humidity levels at {weather_info['humidity']}. Have a great day!"
    ]
    return random.choice(responses)

def search_knowledge(query):
    """Enhanced knowledge search with actual information snippets"""
    knowledge_base = {
        "quantum computing": "Quantum computing uses quantum bits (qubits) that can exist in multiple states simultaneously, enabling them to solve certain problems much faster than classical computers. Key players include IBM, Google, and Microsoft in developing practical quantum systems.",
        
        "artificial intelligence": "AI refers to machines designed to perform tasks that typically require human intelligence. This includes machine learning, natural language processing, computer vision, and robotics. Current trends focus on deep learning and generative AI.",
        
        "renewable energy": "Renewable energy comes from natural sources that replenish faster than consumed. Main types include solar, wind, hydroelectric, geothermal, and biomass. Solar and wind have seen rapid cost reductions making them competitive with fossil fuels.",
        
        "blockchain technology": "Blockchain is a decentralized digital ledger that records transactions across many computers. It's the technology behind cryptocurrencies like Bitcoin and enables smart contracts, NFTs, and decentralized applications.",
        
        "machine learning": "Machine learning is a subset of AI where algorithms improve automatically through experience. Common approaches include supervised learning, unsupervised learning, and reinforcement learning using neural networks and other models."
    }
    
    query_lower = query.lower()
    for topic, info in knowledge_base.items():
        if topic in query_lower:
            responses = [
                f"Here's what I know about {topic}: {info}",
                f"Regarding {topic}: {info}",
                f"Information about {topic}: {info}"
            ]
            return random.choice(responses)
    
    # Generic response for unknown topics
    responses = [
        f"I found some information about '{query}'. This appears to be a topic worth exploring further through specialized resources or recent publications.",
        f"While I don't have detailed specifics about '{query}', I recommend checking authoritative sources for comprehensive information on this subject.",
        f"For detailed information about '{query}', I suggest consulting specialized databases, academic papers, or recent news articles for the most current insights."
    ]
    return random.choice(responses)

def translate_text(text, target_language="Spanish"):
    """Enhanced translation with common phrases"""
    common_translations = {
        "hello": {
            "spanish": "Hola", "french": "Bonjour", "german": "Hallo", 
            "italian": "Ciao", "japanese": "Konnichiwa", "chinese": "Nǐ hǎo"
        },
        "thank you": {
            "spanish": "Gracias", "french": "Merci", "german": "Danke",
            "italian": "Grazie", "japanese": "Arigatō", "chinese": "Xièxiè"
        },
        "how are you": {
            "spanish": "¿Cómo estás?", "french": "Comment allez-vous?", 
            "german": "Wie geht es dir?", "italian": "Come stai?",
            "japanese": "Ogenki desu ka?", "chinese": "Nǐ hǎo ma?"
        },
        "goodbye": {
            "spanish": "Adiós", "french": "Au revoir", "german": "Auf Wiedersehen",
            "italian": "Arrivederci", "japanese": "Sayōnara", "chinese": "Zàijiàn"
        }
    }
    
    text_lower = text.lower().replace('how do you say', '').replace('in', '').strip("? '")
    target_lower = target_language.lower()
    
    for phrase, translations in common_translations.items():
        if phrase in text_lower:
            if target_lower in translations:
                translation = translations[target_lower]
                responses = [
                    f"The translation of '{phrase}' to {target_language} is: '{translation}'",
                    f"In {target_language}, '{phrase}' is said as: '{translation}'",
                    f"'{phrase}' translates to '{translation}' in {target_language}"
                ]
                return random.choice(responses)
    
    # Generic translation response
    responses = [
        f"For accurate translation of '{text}' to {target_language}, I recommend using dedicated translation services like Google Translate or DeepL for precise results.",
        f"While I can help with common phrases, for '{text}' in {target_language}, specialized translation tools will provide the most accurate conversion.",
        f"Translation of '{text}' to {target_language} is best handled by professional translation services to ensure contextual accuracy."
    ]
    return random.choice(responses)

def get_current_time(location):
    """New tool: Get current time for different locations"""
    time_zones = {
        "new york": -4, "london": 1, "tokyo": 9, "paris": 2,
        "sydney": 11, "dubai": 4, "los angeles": -7, "chicago": -5
    }
    
    location_lower = location.lower()
    for loc, offset in time_zones.items():
        if loc in location_lower:
            # Simulate time calculation
            from datetime import datetime, timedelta
            current_time = datetime.utcnow() + timedelta(hours=offset)
            time_str = current_time.strftime("%I:%M %p")
            
            responses = [
                f"Current time in {location.title()}: {time_str}",
                f"The time in {location.title()} is now {time_str}",
                f"In {location.title()}, it's currently {time_str}"
            ]
            return random.choice(responses)
    
    return f" I don't have timezone data for {location}, but you can check world clock websites for accurate time information."

# Enhanced tool selection classifier
tool_prompt = ChatPromptTemplate.from_template("""
Analyze the user's request and classify which tool to use. Choose from:
- math: for calculations, math problems, arithmetic
- weather: for weather inquiries, temperature, climate
- knowledge: for general information searches, facts, explanations
- translate: for language translation, how to say phrases
- time: for current time, time zones, what time is it
- unknown: if no tool matches

Examples:
- "What's 15 * 24?" → math
- "Weather in London" → weather  
- "Tell me about AI" → knowledge
- "How to say hello in French" → translate
- "What time is it in Tokyo" → time

Request: {request}

Respond with only the tool name:
""")

tool_chain = tool_prompt | llm | StrOutputParser()

# Enhanced tool routing logic
def route_to_tool(data):
    request = data["request"]
    tool = tool_chain.invoke({"request": request}).strip().lower()
    
    print(f"Selected tool: {tool}")
    
    # Enhanced tool execution with better context
    if "math" in tool:
        return calculate_math(request)
    elif "weather" in tool:
        # Extract location from request
        location = request.lower().replace('weather', '').replace('in', '').strip()
        return get_weather(location if location else "your location")
    elif "knowledge" in tool:
        return search_knowledge(request)
    elif "translate" in tool:
        # Extract target language
        languages = ["spanish", "french", "german", "italian", "japanese", "chinese"]
        target_lang = "Spanish"  # default
        for lang in languages:
            if lang in request.lower():
                target_lang = lang.title()
                break
        return translate_text(request, target_lang)
    elif "time" in tool:
        # Extract location from time request
        location = request.lower().replace('time', '').replace('what', '').replace('in', '').strip()
        return get_current_time(location if location else "your location")
    else:
        helpful_responses = [
            f"I can help you with: calculations, weather information, general knowledge, translations, and time queries. Your request '{request}' doesn't match my current capabilities perfectly, but I'd be happy to help with related topics!",
            f"I specialize in math, weather, knowledge searches, translations, and time information. For '{request}', maybe try rephrasing or ask about one of these areas?",
            f"My expertise includes: mathematical calculations, weather updates, factual information, language translation, and time zones. How can I assist you within these domains?"
        ]
        return random.choice(helpful_responses)

# Create the enhanced dynamic tool selection chain
tool_selection_chain = RunnableLambda(
    lambda x: {"request": x, "tool_response": route_to_tool({"request": x})}
) | RunnableLambda(lambda x: x["tool_response"])

# Test with diverse and realistic requests
print("ENHANCED TOOL SELECTION SYSTEM")

requests = [
    "What's 15 * 24 + 8?",
    "What's the weather in New York?",
    "Tell me about quantum computing"
    # "How do you say 'thank you' in French?",
    # "What time is it in Tokyo?",
    # "Calculate 45 divided by 9",
    # "What's the temperature in London?",
    # "Explain artificial intelligence",
    # "Translate 'hello' to Japanese",
    # "What's the current time in Paris?",
    # "Can you help me with my homework?",
    # "What's 100 minus 25?"
]

print(f"Testing {len(requests)} different requests...\n")

for i, request in enumerate(requests, 1):
    print(f"#{i}: {request}")
    print("─" * 50)
    try:
        response = tool_selection_chain.invoke(request)
        print(f"{response}\n")
    except Exception as e:
        print(f"Error: {e}\n")

print("Tool selection testing completed!")

ENHANCED TOOL SELECTION SYSTEM
Testing 3 different requests...

#1: What's 15 * 24 + 8?
──────────────────────────────────────────────────
Selected tool: math
Let me calculate that for you: 15 * 24 + 8 = 368

#2: What's the weather in New York?
──────────────────────────────────────────────────
Selected tool: weather
Weather update for What'S The   New York?: Partly Cloudy with temperatures around 85°F. Humidity is 65%. Dress accordingly!

#3: Tell me about quantum computing
──────────────────────────────────────────────────
Selected tool: knowledge
Information about quantum computing: Quantum computing uses quantum bits (qubits) that can exist in multiple states simultaneously, enabling them to solve certain problems much faster than classical computers. Key players include IBM, Google, and Microsoft in developing practical quantum systems.

Tool selection testing completed!
